1. Data preparation

In order to prepare data we have to split them into 3 sets: train, validation and test sets. Train set is used to train a MLP model, and validation is used for evaluate results of models. Test set will be used to compare best MLP with Fuzzy set.

In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

raw_data = pd.read_csv("ACI21-22_Proj1IoTGatewayCrashDataset.csv")
X = raw_data.drop('Falha', axis=1).to_numpy()
y = raw_data['Falha'].to_numpy()

#Splitting the data for train, validation and test set (80%, 20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,  shuffle=False)

Table below presents different hyperparameters used during training of models. Unfortunetly GridSearchCV from sklearn package uses cross validation which can't be used in our example because of sequential data, where order matters. That's why we have to search of hyperparameters by outselfs.

In [45]:
clfs = {
    'adam_2lay': MLPClassifier(hidden_layer_sizes=(2,),max_iter=1000),
    'adam2_8lay': MLPClassifier(hidden_layer_sizes=(8,),max_iter=1000),
    'adam3_32lay': MLPClassifier(hidden_layer_sizes=(32,),max_iter=1000),
    'sgd1_2lay': MLPClassifier(hidden_layer_sizes=(2,),solver='sgd',max_iter=1000, momentum=0),
    'sgd2_8lay': MLPClassifier(hidden_layer_sizes=(8,),solver='sgd',max_iter=1000, momentum=0.9),
    'sgd3_32lay': MLPClassifier(hidden_layer_sizes=(32,),solver='sgd',max_iter=1000, momentum=0.9),
    'sgd1_adaptive_momentum': MLPClassifier(hidden_layer_sizes=(2,),solver='sgd',max_iter=1000, momentum=0),
    'sgd2_momentum': MLPClassifier(hidden_layer_sizes=(8,),solver='sgd',max_iter=1000, momentum=0.9),
    'sgd3_momentum': MLPClassifier(hidden_layer_sizes=(32,),solver='sgd',max_iter=1000, momentum=0.9),
    'lbfgs_no_momentum': MLPClassifier(hidden_layer_sizes=(2,),solver='lbfgs',max_iter=1000, momentum=0),
    'lbfgs_momentum': MLPClassifier(hidden_layer_sizes=(8,),solver='lbfgs',max_iter=1000, momentum=0.9),
    'lbfgs_momentum': MLPClassifier(hidden_layer_sizes=(32,),solver='lbfgs',max_iter=1000, momentum=0.9),
}
def TrainModel(X_train, X_val, y_train, y_val):
    for clf_id, clf_name in enumerate(clfs):
        clf = clfs[clf_name]
        clf.fit(X_train, y_train)
        prediction = clf.predict(X_val)
        print(clf_name)
        print("Confusion matrix(tn, fp, fn, tp):", confusion_matrix(y_val, prediction).ravel())
        print("Accuracy:",accuracy_score(y_val, prediction))
        print("Precision:",precision_score(y_val, prediction,zero_division=0))
        print("Recall:",recall_score(y_val, prediction))

In [31]:
TrainModel(X_train, X_val, y_train, y_val)

C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


adam_2lay
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0


C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


adam2_8lay
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
adam3_32lay
Confusion matrix: [389   0   9   2]
Accuracy: 0.9775
Precision: 1.0
Recall: 0.18181818181818182


C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


sgd1_2lay
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd2_8lay
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd3_32lay
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0


C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


sgd1_adaptive_momentum
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd2_momentum
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd3_momentum
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
lbfgs_no_momentum
Confusion matrix: [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
lbfgs_momentum
Confusion matrix: [388   1   7   4]
Accuracy: 0.98
Precision: 0.8
Recall: 0.36363636363636365


In [86]:
def createPreviousLoadData(n):
    x = raw_data["Load"].to_numpy()
    req = raw_data.drop('Falha', axis=1).to_numpy()
    x = np.vstack((x,req[:,0]))
    for i in range(n):
        requestN =np.roll(req[:,0],i+1)
        print(requestN.shape)
        x = np.vstack((x,requestN))
        
    return x.T

In [89]:
createPreviousLoadData(2)
for i in range(1,5):
    X_train, X_val, y_train, y_val = train_test_split(createPreviousLoadData(i), y, test_size=0.2,  shuffle=False)
    TrainModel(X_train, X_val, y_train, y_val)

(1999,)
(1999,)
(1999,)


C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


adam_2lay
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0


C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


adam2_8lay
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
adam3_32lay
Confusion matrix(tn, fp, fn, tp): [388   1   5   6]
Accuracy: 0.985
Precision: 0.8571428571428571
Recall: 0.5454545454545454


...

adam_2lay
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
adam2_8lay
Confusion matrix(tn, fp, fn, tp): [389   0   4   7]
Accuracy: 0.99
Precision: 1.0
Recall: 0.6363636363636364
adam3_32lay
Confusion matrix(tn, fp, fn, tp): [389   0   2   9]
Accuracy: 0.995
Precision: 1.0
Recall: 0.8181818181818182


C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


sgd1_2lay
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd2_8lay
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd3_32lay
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0


C:\Users\Pjoter\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


sgd1_adaptive_momentum
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd2_momentum
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
sgd3_momentum
Confusion matrix(tn, fp, fn, tp): [389   0  11   0]
Accuracy: 0.9725
Precision: 0.0
Recall: 0.0
lbfgs_no_momentum
Confusion matrix(tn, fp, fn, tp): [388   1   2   9]
Accuracy: 0.9925
Precision: 0.9
Recall: 0.8181818181818182
lbfgs_momentum
Confusion matrix(tn, fp, fn, tp): [388   1   0  11]
Accuracy: 0.9975
Precision: 0.9166666666666666
Recall: 1.0
